In [ ]:
import time
import requests
import pandas as pd
from tqdm import tqdm
import random
from nba_api.stats.endpoints import leaguedashlineups

In [ ]:
data_DIR = "../../data/"
box_DIR = "../../data/box/"
shiny_DIR = data_DIR + "shiny/"
shiny_export_DIR1 = "C:/Users/pansr/Documents/shinyNBA/data/"

## V1

In [ ]:
teams_response = requests.get("https://api.pbpstats.com/get-teams/nba")
teams = teams_response.json()
teams_dict = teams["teams"]
df_teams = pd.DataFrame(teams_dict)
df_teams = df_teams.rename(columns={"text":"team"})
teams_list = df_teams["id"].to_list()
dfa = []
for year in range(2024,2025):
    season = str(year) + '-' + str(year+1)[-2:]
    for team in tqdm(teams_list):
        print(team)
        url = "https://api.pbpstats.com/get-team-players-for-season?S"
        params = {
            "Season": season, # To get for multiple seasons, separate seasons by comma
            "SeasonType": "Regular Season",
            "TeamId": team,
        }
        response = requests.get(url, params=params)
        response_json = response.json()
        players = response_json["players"]
        df_players1 = pd.DataFrame.from_dict(players,orient="index",columns=["player"]).reset_index()
        df_players1 = df_players1.rename(columns={"index":"id"})
        df_players1["team"] = team
        df_players1["season"] = season
        time.sleep(3)
        dfa.append(df_players1)
df_players = pd.concat(dfa)
df_players = pd.merge(df_players,df_teams,left_on="team", right_on="id")
df_players = df_players.rename(columns={"id_x":"pid","id_y":"tid","team_y":"team"})
df_players = df_players.drop(columns=["team_x"]) 
# df_players.to_parquet(shiny_DIR + "lineup_data.parquet")
# df_players.to_parquet(shiny_export_DIR1 + "lineup_data.parquet")

## V2

In [ ]:
df_teams = pd.read_csv(data_DIR+"NBA_teams_database.csv")
df_teams.columns = ["tid","team"]
teams_list = df_teams["tid"].to_list()
df_players = pd.read_parquet(box_DIR + "NBA_Box_P_Base_2024.parquet")
df_players = df_players[['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',]]
df_players.columns = ['pid', 'player', 'tid']
df_players = pd.merge(df_players,df_teams,on="tid")
df_players.to_parquet(shiny_DIR + "lineup_data.parquet")
df_players.to_parquet(shiny_export_DIR1 + "lineup_data.parquet")

In [ ]:
stats = leaguedashlineups.LeagueDashLineups(
    group_quantity=5,
    measure_type_detailed_defense="Base",
    per_mode_detailed="Totals",
    pace_adjust="Y",
    plus_minus="Y",
    rank="Y",
    season_type_all_star="Regular Season"
)
df = stats.get_data_frames()[0]
df.iloc[:,9:]  = df.iloc[:,9:].round(3)

In [ ]:
df1 = df.copy()
df1.columns = map(str.lower,df1.columns)
df1 = df1[['team_id','group_name','gp','min',]]
df1 = df1.rename(columns={"team_id":"tid"})
df2 = pd.merge(df_teams,df1)
df2 = df2.drop(columns=["tid"])
df2.to_parquet(shiny_DIR + "lineup_list_data.parquet")
df2.to_parquet(shiny_export_DIR1 + "lineup_list_data.parquet")

In [ ]:
df2